# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [23]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [24]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [25]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [26]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [27]:
def foldl2(f, x0, lst):
    return foldr(lambda val, acc: lambda n: acc(f(n, val)),  lambda x: x, lst)(x0)

In [28]:
foldl2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [29]:
def foldr2(f, x0, lst):
    return foldl(lambda acc, val: lambda n: acc(f(val, n)), lambda x: x, lst)(x0)

In [30]:
#сумма
foldr2(lambda x, y: x + y, 0, [1, 2, 3])

6

In [31]:
#конкатенация
foldr2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [32]:
f = lambda x, y: x / y
foldr2(f, 1, [1, 2, 3])

1.5

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [33]:
def check_inv(a, b):
    if len(b) > len(a):
        return False

    #Создаем словарь строки b, где ключами являются символы, а значениями - кол-во этих символов в строке
    b_dict = {}
    for c in b:
        val = b_dict.get(c, 0)
        b_dict[c]=val+1

    #Создаем словарь из первых len(b)-1 элементов строки a по такому же принципу
    a_dict = {}
    for c in a[:len(b)-1]:
        val = a_dict.get(c, 0)
        a_dict[c] = val + 1

    #Проходясь по строке а, добавляем следующий символ, проверяем равенство словарей, удаляем левый символ
    for i in range(len(b)-1, len(a)):
        val = a_dict.get(a[i], 0)
        a_dict[a[i]] = val + 1
        if b_dict == a_dict:
            return True
        val = a_dict[a[i-len(b)+1]]
        if val > 1:
            a_dict[a[i-len(b)+1]]= val - 1
        else:
            a_dict.pop(a[i-len(b)+1])
                   
    return False
       

Первый цикл имеет длину len(b): операция get - O(1), операция put - O(1*) => общая сложность: O(len(b))

Второй цикл аналогично - O(len(b)-1)= O(len(b))

Третий цикл имеет длину (len(a) - (len(b) - 1)): сравение словарей, размер которых ограничен размером алфавита => O(1), Операция pop - O(1*) => сложность цикла O(len(a)-len(b))

=> общая сложность функции - O(len(a))

In [34]:
a = 'abcrotm'
b = 'tro'
check_inv(a, b)

True

In [35]:
a = 'obcratm'
b = 'tro'

check_inv(a, b)

False

In [36]:
a = 'rot'
b = 'tro'
check_inv(a, b)

True

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [37]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if self.left == None and self.right == None:
            yield self.value
        if self.left != None:
            for node in self.left:
                yield node
        if self.right != None:
            for node in self.right:
                yield node
    
    def __str__(self):
        #Рекурсивно собираем значения сначала слева, потом справа
        val = [str(self.value)]
        
        if(self.left != None):
            val.append(str(self.left))
            
        if(self.right != None):
            val.append(str(self.right))
        val = ' '.join(val)
        return val
    
    def __repr__(self):
        return 'Tree({value}, {left}, {right})'.format(value = self.value, left = repr(self.left), right = repr(self.right))

tree = Tree(0, Tree(1, Tree(3), Tree(4)), Tree(2))

list(tree) == [3, 4, 2]

True

In [38]:
print(tree)

0 1 3 4 2


In [39]:
tree

Tree(0, Tree(1, Tree(3, None, None), Tree(4, None, None)), Tree(2, None, None))

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [2]:
from operator import add, sub, mul, truediv

def calc(expr):
    
    expr_list = []
    
    #--------------------------------
    #Преобразование строки в список, может использоваться любое количество пробелов
    j = 0
    for i in range(len(expr)):
        t = expr[i]
        if t in ['+', '-', '*', '/', '(', ')']:
            v = expr[j:i].replace(' ', '')
            if len(v)>0:
                expr_list.append(v)
            expr_list.append(t)
            j = i+1
    v = expr[j:].replace(' ', '')
    if len(v)>0:
        expr_list.append(v)
    #---------------------------------------
    
    #--------------------------------------
    #Приведение списка к обратной польской записи
    rpn = []
    operators = []
    i = 0
    priority = {'-':0, '+':0, '*':1, '/':1, '(':0, ')':0}
    for t in expr_list:
        if t.isalnum():
            rpn.append(t)
        elif t in ['+', '-', '*', '/']:
            while len(operators) > 0 and operators[-1] != '(' and (priority[t] < priority[operators[-1]] or priority[t] == priority[operators[-1]]):
                rpn.append(operators.pop())
            operators.append(t)
        elif t == '(':
            operators.append(t)
        elif t == ')':
            #Двигаемся к открывающейся скобке
            while operators[-1] != '(':
                rpn.append(operators.pop())
            if operators[-1] == '(':
                operators.pop()
    rpn += operators
    #-------------------------------------------
    
    #-------------------------------------------
    #Используя стек проводим вычисления
    OP = {'+': add, '-': sub, '*': mul, '/': truediv}
    stack = []
    for c in rpn:
        if c.isalnum():
            stack.append(c)
        else:
            n1, n2 = stack.pop(), stack.pop()
            stack.append(OP[c](int(n2), int(n1)))
    #-------------------------------------------
    
    return stack.pop()


print(calc('2 * (15 - 3 * 4) - 2')  == 4)  
print(calc('10-4/(22-20)+7') == 15)

True
True
